Detect and Count people in zone

In [ ]:
!nvidia-smi

In [ ]:
!pip install torch
import torch
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)

In [ ]:
import os
HOME = os.getcwd()
print(HOME)

## Install YOLOv8

In [ ]:
!pip install ultralytics

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

## Install Supervision

In [ ]:
!pip install supervision==0.2.1


from IPython import display
display.clear_output()


import supervision
print("supervision.__version__:", supervision.__version__)

##DOWNLOAD VIDEO

In [ ]:
!gdown --id 1H1wCn-_IviEOEpoGgb99EbTCBiHAkgl7

In [ ]:
SOURCE_VIDEO_PATH = f"{HOME}/input.mp4"
TARGET_VIDEO_PATH = f"{HOME}/output/output.mp4"
MODEL_NAME = 'yolov8n.pt'

In [ ]:
from ultralytics import YOLO

model = YOLO(MODEL_NAME)

In [ ]:
import supervision as sv

In [ ]:
sv.VideoInfo.from_video_path(SOURCE_VIDEO_PATH)

In [ ]:
from typing import Optional, Tuple

import cv2
import numpy as np
import supervision
from supervision import Detections
from supervision.detection.utils import generate_2d_mask
from supervision.draw.color import Color
from supervision.draw.utils import draw_polygon, draw_text
from supervision.geometry.core import Position
from supervision.geometry.utils import get_polygon_center
# from google.colab.patches import cv2_imshow

class PolygonZone:
    def __init__(
        self,
        polygon: np.ndarray,
        frame_resolution_wh: Tuple[int, int],
        triggering_position: Position = Position.BOTTOM_CENTER,
    ):
        self.polygon = polygon
        self.frame_resolution_wh = frame_resolution_wh
        self.triggering_position = triggering_position
        self.mask = generate_2d_mask(
            polygon=polygon, resolution_wh=frame_resolution_wh)
        self.current_count = 0

    def trigger(self, detections, data) -> np.ndarray:
        anchors = (
            np.ceil(
                detections.get_anchor_coordinates(
                    anchor=self.triggering_position)
            ).astype(int)
            - 1
        )
        is_in_zone = self.mask[anchors[:, 1], anchors[:, 0]]
        self.current_count = np.sum(is_in_zone)
        data.append(self.current_count)
        return is_in_zone.astype(bool)


class PolygonZoneAnnotator:
    def __init__(
        self,
        zone: PolygonZone,
        color: Color,
        thickness: int = 2,
        text_color: Color = Color.black(),
        text_scale: float = 0.5,
        text_thickness: int = 1,
        text_padding: int = 10,
    ):
        self.zone = zone
        self.color = color
        self.thickness = thickness
        self.text_color = text_color
        self.text_scale = text_scale
        self.text_thickness = text_thickness
        self.text_padding = text_padding
        self.font = cv2.FONT_HERSHEY_SIMPLEX
        self.center = get_polygon_center(polygon=zone.polygon)

    def annotate(self, scene: np.ndarray, label: Optional[str] = None) -> np.ndarray:
        annotated_frame = draw_polygon(
            scene=scene,
            polygon=self.zone.polygon,
            color=self.color,
            thickness=self.thickness,
        )

        annotated_frame = draw_text(
            scene=annotated_frame,
            text=str(self.zone.current_count) if label is None else label,
            text_anchor=self.center,
            background_color=self.color,
            text_color=self.text_color,
            text_scale=self.text_scale,
            text_thickness=self.text_thickness,
            text_padding=self.text_padding,
            text_font=self.font,
        )

        return annotated_frame

In [ ]:
import numpy as np
import supervision as sv
import cv2

from typing import Optional, Tuple

import cv2
import numpy as np
import supervision
from supervision import Detections
from supervision.detection.utils import generate_2d_mask
from supervision.draw.color import Color
from supervision.draw.utils import draw_polygon, draw_text
from supervision.geometry.core import Position
from supervision.geometry.utils import get_polygon_center
# from google.colab.patches import cv2_imshow

class PolygonZone:
    def __init__(
        self,
        polygon: np.ndarray,
        frame_resolution_wh: Tuple[int, int],
        triggering_position: Position = Position.BOTTOM_CENTER,
    ):
        self.polygon = polygon
        self.frame_resolution_wh = frame_resolution_wh
        self.triggering_position = triggering_position
        self.mask = generate_2d_mask(
            polygon=polygon, resolution_wh=frame_resolution_wh)
        self.current_count = 0

    def trigger(self, detections, data) -> np.ndarray:
        anchors = (
            np.ceil(
                detections.get_anchor_coordinates(
                    anchor=self.triggering_position)
            ).astype(int)
            - 1
        )
        is_in_zone = self.mask[anchors[:, 1], anchors[:, 0]]
        self.current_count = np.sum(is_in_zone)
        data.append(self.current_count)
        return is_in_zone.astype(bool)


class PolygonZoneAnnotator:
    def __init__(
        self,
        zone: PolygonZone,
        color: Color,
        thickness: int = 2,
        text_color: Color = Color.black(),
        text_scale: float = 0.5,
        text_thickness: int = 1,
        text_padding: int = 10,
    ):
        self.zone = zone
        self.color = color
        self.thickness = thickness
        self.text_color = text_color
        self.text_scale = text_scale
        self.text_thickness = text_thickness
        self.text_padding = text_padding
        self.font = cv2.FONT_HERSHEY_SIMPLEX
        self.center = get_polygon_center(polygon=zone.polygon)

    def annotate(self, scene: np.ndarray, label: Optional[str] = None) -> np.ndarray:
        annotated_frame = draw_polygon(
            scene=scene,
            polygon=self.zone.polygon,
            color=self.color,
            thickness=self.thickness,
        )

        annotated_frame = draw_text(
            scene=annotated_frame,
            text=str(self.zone.current_count) if label is None else label,
            text_anchor=self.center,
            background_color=self.color,
            text_color=self.text_color,
            text_scale=self.text_scale,
            text_thickness=self.text_thickness,
            text_padding=self.text_padding,
            text_font=self.font,
        )

        return annotated_frame



data = [0, 0]
res = 0


def process_frame(frame: np.ndarray, _) -> np.ndarray:
    # 3 detect
    results = model(frame, imgsz=1280)[0]
    detections = sv.Detections.from_yolov8(results)
    detections = detections[(detections.class_id == 0)
                            & (detections.confidence > 0.4)]
    zone.trigger(detections)

    # 4 annotate
    box_annotator = sv.BoxAnnotator(
        thickness=4, text_thickness=4, text_scale=2)
    labels = [f"{model.names[class_id]} {confidence:0.2f}" for _,
              confidence, class_id, _ in detections]
    frame = box_annotator.annotate(
        scene=frame, detections=detections, labels=labels)
    frame = zone_annotator.annotate(scene=frame)

    return frame


# 1 initiate polygon zone
polygon = np.array([
    [961, 1078],
    [763, 550],
    [1502, 254],
    [1914, 260],
    [1920, 1080]
])
video_info = sv.VideoInfo.from_video_path(SOURCE_VIDEO_PATH)
zone = sv.PolygonZone(
    polygon=polygon, frame_resolution_wh=video_info.resolution_wh)

# 2 initiate annotators
box_annotator = sv.BoxAnnotator(thickness=4, text_thickness=4, text_scale=2)
zone_annotator = sv.PolygonZoneAnnotator(
    zone=zone, color=sv.Color.white(), thickness=6, text_thickness=6, text_scale=4)


# Xuat ket qua ra file
sv.process_video(source_path=SOURCE_VIDEO_PATH,
                 target_path=TARGET_VIDEO_PATH, callback=process_frame)
display.clear_output()


# # Xuat ra man hinh
# # extract video frame
# generator = sv.get_video_frames_generator(SOURCE_VIDEO_PATH)
# iterator = iter(generator)
# frame = next(iterator)

# # matrix_rotate = np.array([[0.9397, 0.342], [-0.342, 0.9397]])
# temp = [0, 0]
# i = 0
# sumPerson = 0
# currentLen = 0
# for frame in iterator:

#     # frame = np.dot(frame, matrix_rotate)
#     current = process_frame(frame, 1)

#     if i < 10:
#         sumPerson += data[-1]
#         i += 1
#     else:
#         soNguoiTrong10Frame = sumPerson/10
#         if soNguoiTrong10Frame-soNguoiTrong10Frame//1 < 0.5:
#             temp.append(soNguoiTrong10Frame//1)
#         else:
#             temp.append(soNguoiTrong10Frame//1+1)
#         currentLen += 1
#         sumPerson = 0
#         i = 0

#         print(temp)
#         currentt = temp[-1]
#         previous = temp[-2]
#         if currentt < previous:
#             res = max(res, previous)
#         else:
#             res += currentt-previous
#         print(res)
#     cv2_imshow(current)
#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         break

# cv2.destroyAllWindows()

